In [ ]:
!pip install grad-cam

### Imports

In [ ]:
import torch
import torchvision
import os
import json
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from torchvision import models
import torch.nn.functional as F
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
cudnn.benchmark = True  # fire on all cylinders
from sklearn.metrics import roc_auc_score, roc_curve
from copy import copy
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import random
from PIL import Image, ImageFilter
import urllib
import matplotlib.cm as cm
from collections import Sequence
import matplotlib
from torchvision.transforms import Compose, Normalize, ToTensor
from typing import List, Dict
import math


import cv2
from pytorch_grad_cam import EigenCAM, EigenGradCAM, LayerCAM
from pytorch_grad_cam import GradCAM, GradCAMPlusPlus, GradCAMElementWise
from pytorch_grad_cam import AblationCAM, RandomCAM, FullGrad, ScoreCAM, HiResCAM, XGradCAM
from pytorch_grad_cam.guided_backprop import GuidedBackpropReLUModel
from pytorch_grad_cam.utils.image import show_cam_on_image, scale_cam_image

import requests

In [ ]:
# img_url_ = r'https://cdn.psychologytoday.com/sites/default/files/styles/article-inline-half-caption/public/field_blog_entry_images/2020-04/cb.jpg?itok=zzuVtGPr'
img_url = r'https://www.purina.co.uk/sites/default/files/styles/ttt_image_510/public/2020-11/Should%20I%20Get%20a%20Cat%20or%20Dog1.jpg?itok=IdntHkbV'
img_name = 'dog.jpeg'
urllib.request.urlretrieve(img_url, img_name)
# image = "/home/results/korgi.jpeg"
image = img_name
device = 'cuda'

### Util functions 

In [ ]:
def im_show(im_path):
    
    img = np.array(Image.open(im_path))
    img = cv2.resize(img, (224, 224))
    rgb_img = img.copy()
    img = np.float32(img) / 255
    plt.imshow(img)
    
    return img

def load_images(image_paths):
    images = []
    raw_images = []
    print("Images:")
    for i, image_path in enumerate([image_paths]):
        print("\t#{}: {}".format(i, image_path))
        
        image, raw_image = preprocess(image_path)
        images.append(image)
        raw_images.append(raw_image)
    return images, raw_images

def get_device(cuda):
    cuda = cuda and torch.cuda.is_available()
    device = torch.device("cuda" if cuda else "cpu")
    if cuda:
        current_device = torch.cuda.current_device()
        print("Device:", torch.cuda.get_device_name(current_device))
    else:
        print("Device: CPU")
    return device

def save_sensitivity(filename, maps):
    maps = maps.cpu().numpy()
    scale = max(maps[maps > 0].max(), -maps[maps <= 0].min())
    maps = maps / scale * 0.5
    maps += 0.5
    maps = cm.bwr_r(maps)[..., :3]
    maps = np.uint8(maps * 255.0)
    maps = cv2.resize(maps, (224, 224), interpolation=cv2.INTER_NEAREST)
    cv2.imwrite(filename, maps)
    
def preprocess(image_path):
    raw_image = cv2.imread(image_path)
    # print(raw_image.shape)
    raw_image = cv2.resize(raw_image, (224, 224))
    image = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )(raw_image[..., ::-1].copy())
    return image, raw_image


### Interpretation-functions
**Common methods**
- Saliency maps - done
- Occlussion sensativity - done
- Integrated Gradients - **should be tested**
- LRPs
- Deep Taylor Decomposition

**Backprops**
- VanilaBackprop
- GuidedBackprop - done

**CAMs**
- GradCAM - done
- HiResCAM - done
- GradCAM ElementWise - done
- GradCAM++ - done
- XGradCAM - done
- AblationCAM - done
- ScoreCAM - done
- EigenCAM - done
- EigenGradCAM - done
- LayerCAM- done 
- RandomCAM - done
- FullGrad - done
- Deep Feature Factorizations

**Global methods**
- SHAP
- LIME

In [ ]:
def eigen_cam_gen(model, img, target_layers):
    transform = transforms.ToTensor()
    tensor = transform(img).unsqueeze(0)
    tensor.to(device)
    cam = EigenCAM(model, target_layers, use_cuda= True)
    grayscale_cam = cam(tensor)[0, :, :]
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
    return cam_image, grayscale_cam

def eigengrad_cam_gen(model, img, target_layers):
    transform = transforms.ToTensor()
    tensor = transform(img).unsqueeze(0)
    tensor.to(device)
    cam = EigenGradCAM(model, target_layers, use_cuda= True)
    grayscale_cam = cam(tensor)[0, :, :]
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
    return cam_image, grayscale_cam

def grad_cam_gen(model, img, target_layers):
    transform = transforms.ToTensor()
    tensor = transform(img).unsqueeze(0)
    tensor.to(device)
    cam = GradCAM(model, target_layers, use_cuda= True)
    grayscale_cam = cam(tensor)[0, :, :]
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
    return cam_image, grayscale_cam

def gradpp_cam_gen(model, img, target_layers):
    transform = transforms.ToTensor()
    tensor = transform(img).unsqueeze(0)
    tensor.to(device)
    cam = GradCAM(model, target_layers, use_cuda= True)
    grayscale_cam = cam(tensor)[0, :, :]
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
    return cam_image, grayscale_cam

# --------

def ablation_cam_gen(model, img, target_layers):
    transform = transforms.ToTensor()
    tensor = transform(img).unsqueeze(0)
    tensor.to(device)
    cam = AblationCAM(model, target_layers, use_cuda= True)
    grayscale_cam = cam(tensor)[0, :, :]
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
    return cam_image, grayscale_cam

def random_cam_gen(model, img, target_layers):
    transform = transforms.ToTensor()
    tensor = transform(img).unsqueeze(0)
    tensor.to(device)
    cam = RandomCAM(model, target_layers, use_cuda= True)
    grayscale_cam = cam(tensor)[0, :, :]
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
    return cam_image, grayscale_cam

def fullgrad_cam_gen(model, img, target_layers):
    transform = transforms.ToTensor()
    tensor = transform(img).unsqueeze(0)
    tensor.to(device)
    cam = FullGrad(model, target_layers, use_cuda= True)
    grayscale_cam = cam(tensor)[0, :, :]
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
    return cam_image, grayscale_cam

def score_cam_gen(model, img, target_layers):
    transform = transforms.ToTensor()
    tensor = transform(img).unsqueeze(0)
    tensor.to(device)
    cam = ScoreCAM(model, target_layers, use_cuda= True)
    grayscale_cam = cam(tensor)[0, :, :]
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
    return cam_image, grayscale_cam

def hires_cam_gen(model, img, target_layers): # need test
    transform = transforms.ToTensor()
    tensor = transform(img).unsqueeze(0)
    tensor.to(device)
    cam = HiResCAM(model, target_layers, use_cuda= True)
    grayscale_cam = cam(tensor)[0, :, :]
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
    return cam_image, grayscale_cam

def elw_grad_cam_gen(model, img, target_layers): # need test
    transform = transforms.ToTensor()
    tensor = transform(img).unsqueeze(0)
    tensor.to(device)
    cam = GradCAMElementWise(model, target_layers, use_cuda= True)
    grayscale_cam = cam(tensor)[0, :, :]
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
    return cam_image, grayscale_cam

def xgrad_cam_gen(model, img, target_layers): # need test
    transform = transforms.ToTensor()
    tensor = transform(img).unsqueeze(0)
    tensor.to(device)
    cam = XGradCAM(model, target_layers, use_cuda= True)
    grayscale_cam = cam(tensor)[0, :, :]
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
    return cam_image, grayscale_cam

def layer_cam_gen(model, img, target_layers): # need test
    transform = transforms.ToTensor()
    tensor = transform(img).unsqueeze(0)
    tensor.requires_grad_()
    tensor.to(device)
    cam = LayerCAM(model, target_layers, use_cuda= True)
    grayscale_cam = cam(tensor)[0, :, :]
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
    return cam_image, grayscale_cam
#-----------

def guided_backprop_gen(model, img, target_layers):
    transform = transforms.ToTensor()
    tensor = transform(img).unsqueeze(0)
    tensor.to(device)
    cam = GuidedBackpropReLUModel(model, target_layers)
    grayscale_cam = cam(tensor)[:, :, 0]
    print(grayscale_cam.shape)
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
    return cam_image, grayscale_cam

def saliency_gen(img, model):
    # we don't need gradients weights for a trained model
    for param in model.parameters():
        param.requires_grad = False

    model.eval()
    input = img
    input.unsqueeze_(0)
    input.requires_grad = True
    preds = model(input)
    score, indices = torch.max(preds, 1)
    score.backward()
    slc, _ = torch.max(torch.abs(input.grad[0]), dim=0)

    return slc


# Integrated gradients should be updated. TODO: issue with "grad" 
def integrated_grads_gen(batch_x, model, batch_blank_type= 'zero', iterations = 100):
    mean_grad = 0
    
    transform = transforms.ToTensor()
    batch_x = transform(batch_x).unsqueeze(0)
    batch_x = batch_x.to(device)
    model.to(device)
    
    if batch_blank_type == 'zero':
        batch_blank = torch.zeros_like(batch_x)
    elif batch_blank_type == 'one':
        batch_blank = torch.ones_like(batch_x)
    elif batch_blank_type == 'rand':
        batch_blank = torch.rand_like(batch_x)
    
    batch_blank = batch_blank.to(device)
    
    for i in tqdm(range(1, iterations + 1)):
        k = i / iterations
        x = batch_blank + k * (batch_x - batch_blank)
        x = Variable(x, requires_grad = True)
        x = x.to(device)
        
        with torch.enable_grad():
            outputs = model(x)
            
            value, preds = torch.max(outputs, 1)

            print(value, preds)
            predictions = preds.type(torch.cuda.FloatTensor)
            predictions = Variable(predictions, requires_grad = True)
            predictions.retain_grad()
            
            # Comment underline is a 1st approach to get grads
            # predictions.backward(retain_graph=True)
            # print(x.grad)
            # grad = x.grad
            
            # Comment underline is a 2nd approach to get grads
            (grad,) = torch.autograd.grad(predictions, x,  allow_unused = True)
            
            # grad = grad.retain_grad()
        
         
        print('iter = ', i, 'grad ', grad)
        if grad == None:
            grad = 0
        mean_grad += grad / iterations

    integrated_gradients = (batch_x - batch_blank) * mean_grad

    return integrated_gradients, mean_grad

def occlusion_sensitivity(model, 
                          images, 
                          ids, 
                          mean=None, 
                          patch=35, 
                          stride=1, 
                          n_batches=128):

    torch.set_grad_enabled(False)
    model.eval()
    mean = mean if mean else 0
    patch_H, patch_W = patch if isinstance(patch, Sequence) else (patch, patch)
    pad_H, pad_W = patch_H // 2, patch_W // 2

    # Padded image
    images = F.pad(images, (pad_W, pad_W, pad_H, pad_H), value=mean)
    B, _, H, W = images.shape
    new_H = (H - patch_H) // stride + 1
    new_W = (W - patch_W) // stride + 1

    # Prepare sampling grids
    anchors = []
    grid_h = 0
    while grid_h <= H - patch_H:
        grid_w = 0
        while grid_w <= W - patch_W:
            grid_w += stride
            anchors.append((grid_h, grid_w))
        grid_h += stride

    # Baseline score without occlusion
    baseline = model(images).detach().gather(1, ids)

    # Compute per-pixel logits
    scoremaps = []
    for i in tqdm(range(0, len(anchors), n_batches), leave=False):
        batch_images = []
        batch_ids = []
        for grid_h, grid_w in anchors[i : i + n_batches]:
            images_ = images.clone()
            images_[..., grid_h : grid_h + patch_H, grid_w : grid_w + patch_W] = mean
            batch_images.append(images_)
            batch_ids.append(ids)
        batch_images = torch.cat(batch_images, dim=0)
        batch_ids = torch.cat(batch_ids, dim=0)
        scores = model(batch_images).detach().gather(1, batch_ids)
        scoremaps += list(torch.split(scores, B))

    diffmaps = torch.cat(scoremaps, dim=1) - baseline
    diffmaps = diffmaps.view(B, new_H, new_W)

    return diffmaps


def occlusion_sens_gen(image_paths, class_names, model, output_dir, cuda, topk, stride, n_batches):
    
    device = get_device(cuda)
    classes = class_names

    # Model from torchvision
    model = model
    model = torch.nn.DataParallel(model)
    model.to(device)
    model.eval()

    # Images
    images, _ = load_images(image_paths)
    images = torch.stack(images).to(device)

    print("Occlusion Sensitivity:")

    patche_sizes = [10, 15, 50]

    logits = model(images)
    probs = F.softmax(logits, dim=1)
    probs, ids = probs.sort(dim=1, descending=True)

    for i in range(topk):
        for p in patche_sizes:
            print("Patch:", p)
            sensitivity = occlusion_sensitivity(
                model, images, ids[:, [i]], patch=p, stride=stride, n_batches=n_batches
            )

            # Save results as image files
            for j in range(len(images)):
                print("\t#{}: {} ({:.5f})".format(j, classes[ids[j, i]], probs[j, i]))

                save_sensitivity(
                    filename=os.path.join(
                        output_dir, "new"+str(i)+".png"
                    ),
                    maps=sensitivity[j],
                )

def LRP(model, img, lrp_type = ['zero']):
    pass

def connctivity_prop(model, img):
    pass

### Model init and applying interpretation

In [ ]:
model = models.resnet18(pretrained= True)

In [ ]:
# # Occlussion sens function calling
# path_ = image
# out_dir='./'

# classes = [i for i in range(1000)]

# occlusion_sens_gen(path_, classes, model, out_dir, cuda=1, topk= 5, stride=1, n_batches=128)

In [ ]:
  # All methods without occlussion function calling

res, gray_res = layer_cam_gen(model, 
                               im_show(image),
                               target_layers= [list(model.children())[1]])


In [ ]:
plt.imshow(res)

In [ ]:
from typing import List, Callable

def tensor_x_cam(input_tensor, cam):
  return input_tensor * cam

def scores_on_targets(targets: List[Callable], input: torch.Tensor):
    with torch.no_grad():
      logits = model(input)
      scores = [target(output).cpu().numpy() for target, output in zip(targets, logits)]
      return np.float32(scores)

def confidence_change_apply_cam(input_tensor: torch.Tensor, grayscale_cams: np.ndarray, targets: List[Callable], model, return_diff=True):
  modified = []
  for i in range(input_tensor.size(0)):
      tensor = tensor_x_cam(input_tensor[i, ...].cpu(), torch.from_numpy(grayscale_cams[i]))
      tensor = tensor.to(input_tensor.device)
      modified.append(tensor.unsqueeze(0))
  modified = torch.cat(modified)

  scores_on_modified = scores_on_targets(targets = targets, input = modified)
  scores_raw = scores_on_targets(targets = targets, input = input_tensor)
  scores_difference = scores_on_modified - scores_raw

  return scores_raw,scores_on_modified, scores_difference, modified



def view(input_tensor, modified_tensor, scores_difference, preservation=False):
    input_tens = input_tensor.detach().clone() 
    modified_tens = modified_tensor.detach().clone() 

    input_tens = input_tens.squeeze(0).cpu().numpy().transpose((1, 2, 0))
    modified_tens = modified_tens.squeeze(0).cpu().numpy().transpose((1, 2, 0))

    print(f"Confidence change: {100*scores_difference} %")
    raw_img = Image.fromarray(deprocess_image(input_tens))
    modified_img = Image.fromarray(deprocess_image(modified_tens))

    subplots = [
        ('Source img', [(raw_img, None, None)]),
        ('Source img & Saliency mapping', [(modified_img, None, None)])
    ]
    if preservation:
          subplots = [
        ('Before_addition', [(raw_img, None, None)]),
        ('After addition', [(modified_img, None, None)])
    ]

    num_subplots = len(subplots)

    fig = plt.figure(figsize=(16, 3))

    for i, (title, images) in enumerate(subplots):
        ax = fig.add_subplot(1, num_subplots, i + 1)
        ax.set_axis_off()
        for image, cmap, alpha in images:
            ax.imshow(image, cmap=cmap, alpha=alpha)
        ax.set_title(title)

def get_target_index(model, input_tensor):
  model.eval()
  out = model(input_tensor.cuda())
  _, index = torch.max(out, 1)
  return out, index

def target_outs(model, input_tensor, softmax=True):
  #TODO: softmax targets
  out, index = get_target_index(model, input_tensor)
  if len(out.shape) == 1:
    if softmax:
      return torch.softmax(out, dim=-1)[index]
    else:
      return out[index]
  else:
    if softmax:
      return  torch.softmax(out, dim=-1)[:, index]
    else:
      return out[:, index]

def deprocess_image(img):
    img = 0.1 * (img - np.mean(img)) / (np.std(img) + 1e-5) + 0.5
    img = np.clip(img, 0, 1)
    return np.uint8(img * 255)

def preprocess_image(
    img: np.ndarray, mean=[
        0.5, 0.5, 0.5], std=[
            0.5, 0.5, 0.5]) -> torch.Tensor:
    preprocessing = Compose([
        ToTensor(),
        Normalize(mean=mean, std=std)
    ])
    return preprocessing(img.copy()).unsqueeze(0)

class ClassifierOutputSoftmaxTarget:
    def __init__(self, category):
        self.category = category

    def __call__(self, model_output):
        if len(model_output.shape) == 1:
            return torch.softmax(model_output, dim=-1)[self.category]
        return torch.softmax(model_output, dim=-1)[:, self.category]

# class ClassifierOutputTarget:
#     def __init__(self, category):
#         self.category = category

#     def __call__(self, model_output):
#         if len(model_output.shape) == 1:
#             return model_output[self.category]
#         return model_output[:, self.category]




In [ ]:
model = models.resnet18(pretrained= True)

In [ ]:
img_name = 'dog.jpeg'
urllib.request.urlretrieve(img_url, img_name)
image = img_name


In [ ]:
image_, raw_image = preprocess(image)

In [ ]:
res, gray_res = guided_backprop_gen(model, 
                               im_show(image),
                              # [list(model.children())[2]],
                               target_layers= [model.layer4],
                              # target_layers = [ClassifierOutputSoftmaxTarget(219)],
                              )

In [ ]:
res, gray_res = layer_cam_gen(model, 
                               im_show(image),
                              # [list(model.children())[2]],
                               target_layers= [model.layer4],
                              # target_layers = [ClassifierOutputSoftmaxTarget(219)],
                              )

In [ ]:
plt.imshow(res)

In [ ]:
plt.imshow(gray_res)

In [ ]:
np.average(gray_res)

In [ ]:
img = np.array(Image.open(requests.get(img_url, stream=True).raw))
print(img.shape)
img = cv2.resize(img, (224, 224))
img = np.float32(img) / 255
input_tensor = preprocess_image(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])


In [ ]:
out, index = get_target_index(model, input_tensor)

In [ ]:
targets = [ClassifierOutputSoftmaxTarget(index[0])]
gray_res *= 100
scores_raw, scores_on_modified, scores_difference, modified = confidence_change_apply_cam(input_tensor.cuda(), gray_res, targets, model)
view(input_tensor=input_tensor, modified_tensor=modified, scores_difference=scores_difference[0])

In [ ]:
def deletion_metric(input_tensor, cam, model, percentile, outs=True):
  '''change something above nth percentile with zeros, check confidence change'''
  new_cam = np.where(cam > np.percentile(cam, percentile), 0, cam)
  out, index = get_target_index(model, input_tensor)
  targets = [ClassifierOutputSoftmaxTarget(index[0])]
  score_raw, scores_on_modified, scores_difference, modified = confidence_change_apply_cam(input_tensor.cuda(), new_cam, targets, model)
  if outs:
    print("...On deletion:", scores_on_modified)
    view(input_tensor=input_tensor, modified_tensor=modified, scores_difference=scores_on_modified)
  return new_cam, scores_on_modified, modified

In [ ]:
d, _, _ = deletion_metric(input_tensor.cuda(), gray_res, model, percentile=30)

In [ ]:
def deletion_game(input_tensor, cam, model):
    fin_array = []
    for perc in range(10000, 0, -10):
      perc = perc/100
      cam_, diff_, modified = deletion_metric(input_tensor.cuda(), gray_res, model, percentile=perc, outs=False)
      fin_array.append([diff_, perc, cam_, modified])
      fin_array.sort(key=lambda row: (row[0], row[1]), reverse=True)
    diff_, perc, cam_, modified = fin_array[0]
    print("...On deletion game:")
    view(input_tensor=input_tensor, modified_tensor=modified, scores_difference=diff_)
    return diff_, perc, cam_


In [ ]:
diff_, perc, cam_ = deletion_game(input_tensor, cam=gray_res, model=model)


In [ ]:
def preservation_metric(input_tensor, cam, model, left_percentile, right_percentile,outs=True):
  '''change something above nth percentile with zeros, check confidence change'''
  new_cam_left = np.where(cam < np.percentile(cam, left_percentile), cam, 0)
  new_cam_right = np.where(cam < np.percentile(cam, right_percentile), cam, 0)
  out, index = get_target_index(model, input_tensor)
  targets = [ClassifierOutputSoftmaxTarget(index[0])]
  score_raw_left, scores_on_modified_left, scores_difference_left, modified_left = confidence_change_apply_cam(input_tensor.cuda(), new_cam_left, targets, model)
  score_raw_right, scores_on_modified_right, scores_difference_right, modified_right = confidence_change_apply_cam(input_tensor.cuda(), new_cam_right, targets, model)
  scores_difference = scores_on_modified_right[0] - scores_on_modified_left[0]

  if outs:
    print(score_raw_left, scores_on_modified_left, scores_difference_left)
    print(score_raw_right, scores_on_modified_right, scores_difference_right)
    print("...On addition:")
    view(input_tensor=modified_left, modified_tensor=modified_right, scores_difference=scores_difference, preservation=True)
  return scores_difference, modified_left, modified_right, new_cam_left, new_cam_right

In [ ]:
_, a,b,_,_ = preservation_metric(input_tensor.cuda(), gray_res, model, left_percentile=12, right_percentile=100)

In [ ]:
def preservation_game(input_tensor, cam, model):
    fin_array = []
    eps = 0.01
    for perc in range(10, 10000, 10):
      perc_l, perc_r = (perc - 10)*eps, perc*eps
      scores_difference, new_cam_left, new_cam_right,_,_ = preservation_metric(input_tensor, cam, model, perc_l, perc_r,outs=False)
      fin_array.append([perc_r, scores_difference, new_cam_left, new_cam_right])
      fin_array.sort(key=lambda row: (row[1]), reverse=True)
    perc_r, diff_, modified_left, modified_right = fin_array[0]
    print("...On preservation game:")
    view(input_tensor=input_tensor, modified_tensor=modified_right, scores_difference=diff_, preservation=True)
    return diff_, perc, cam_

In [ ]:
pres_game = preservation_game(input_tensor, cam=gray_res, model=model)

In [ ]:
list_tensors_ = [input_tensor, ]
list_cams_ = [gray_res, ]

In [ ]:
def average_drop_item(input_tensor, cam, model, index=219):
  targets = [ClassifierOutputSoftmaxTarget(index)]
  score_raw, scores_on_modified, scores_difference, modified = confidence_change_apply_cam(input_tensor.cuda(), cam, targets, model)
  return max(0,-scores_difference[0])*100/score_raw[0]

In [ ]:
average_drop_item(input_tensor, gray_res, model)

In [ ]:
def avg_drop_list(list_tens, list_cam, model, index=219):
  list_out = []
  for tensor, cam in zip(list_tens, list_cam):
    list_out.append(average_drop_item(tensor, cam, model, index))
  return np.average(list_out)

In [ ]:
avg_drop_list(list_tens=list_tensors_, list_cam=list_cams_, model=model, index=219)
#одинаково тк одно изображение в массиве

In [ ]:
def increase_in_confidence_item(input_tensor, cam, model, index=219):
  targets = [ClassifierOutputSoftmaxTarget(index)]
  score_raw, scores_on_modified, scores_difference, modified = confidence_change_apply_cam(input_tensor.cuda(), cam, targets, model)
  if scores_difference[0] > 0:
    return 1
  return 0

In [ ]:
increase_in_confidence_item(input_tensor, gray_res, model)

In [ ]:
def increase_in_confidence_list(list_tens, list_cam, model, index=219):
  list_out = []
  for tensor, cam in zip(list_tens, list_cam):
    list_out.append(increase_in_confidence_item(tensor, cam, model, index))
  return np.average(list_out)

In [ ]:
increase_in_confidence_list(list_tens=list_tensors_, list_cam=list_cams_, model=model, index=219)
#одинаково тк одно изображение в массиве

###SPARSITY

In [ ]:
gr_r = gray_res.copy()

In [ ]:
gr_r_norm = gr_r - np.min(gr_r)/ np.max(gr_r) - np.min(gr_r)

In [ ]:
def sparsity(cam):
  gr_r = cam.copy()
  gr_r_norm = gr_r - np.min(gr_r)/ np.max(gr_r) - np.min(gr_r)
  return 1/np.mean(gr_r_norm)

print(f" Sparsity = {sparsity(cam=gray_res)}")